In [ ]:
'''
simple ablation study
'''

In [2]:
# general imports
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pytorch_utils import Conv1D
import torch
import torch.nn as nn
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import json


# another code we wrote
sys.path.append('../')
import llm_utils
import opt_utils
from plot_utils import plot_aux_wrapper
import exp_ra_utils

/Users/ks/miniconda3/envs/april24/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


llm_utils.py: sys.path: ['/Users/ks/Documents/research/projD2/opt07/bl2/clean_bu_code_v2', '/Users/ks/miniconda3/envs/april24/lib/python310.zip', '/Users/ks/miniconda3/envs/april24/lib/python3.10', '/Users/ks/miniconda3/envs/april24/lib/python3.10/lib-dynload', '', '/Users/ks/miniconda3/envs/april24/lib/python3.10/site-packages', '/Users/ks/Documents/research/projD2/opt07/bl2/clean_bu_code_v2', '/Users/ks/Documents/research/projD2/opt07/bl2']
llm_utils.py: sys.path: ['/Users/ks/Documents/research/projD2/opt07/bl2/clean_bu_code_v2', '/Users/ks/miniconda3/envs/april24/lib/python310.zip', '/Users/ks/miniconda3/envs/april24/lib/python3.10', '/Users/ks/miniconda3/envs/april24/lib/python3.10/lib-dynload', '', '/Users/ks/miniconda3/envs/april24/lib/python3.10/site-packages', '/Users/ks/Documents/research/projD2/opt07/bl2/clean_bu_code_v2', '/Users/ks/Documents/research/projD2/opt07/bl2', '/Users/ks/Documents/research/projD2/opt07/bl2']
'NoneType' object has no attribute 'cadam32bit_grad_fp32'

/Users/ks/miniconda3/envs/april24/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
try:
    from function_vectors.src.utils.prompt_utils import load_dataset, word_pairs_to_prompt_data, create_prompt
    from function_vectors.src.utils.eval_utils import decode_to_vocab, sentence_eval, n_shot_eval_no_intervention, get_answer_id
except Exception as error:
    print('could not import from function_vectors package with the following error:')
    print(error)
    print('Make sure you first pull relevant submodules. See README.md for more info.')

In [4]:
START_TIME = time.strftime("%Y/%m/%d-%H:%M:%S")
# DEBUG_FLAG = True  # for local testing
DEBUG_FLAG = False  # automatically set below

try:
    DEBUG_FLAG = torch.backends.mps.is_available()  # since only Apple M1-3 supports this, we can use this as a flag for local testing
except:
    pass
if DEBUG_FLAG:
    print('*'*40)
    print(f'    DEBUG MODE [{START_TIME}]')
    print('*'*40)

****************************************
    DEBUG MODE [2024/09/28-17:11:12]
****************************************


In [5]:
import argparse
parser = argparse.ArgumentParser()
# parser.add_argument('--model_name', type=str, default='facebook/opt-350m')
parser.add_argument('--model_name', type=str, default='gpt2' if DEBUG_FLAG else 'gpt2-xl')
parser.add_argument('--model_args', type=str, default='')
parser.add_argument('--out_folder', type=str, default='/Users/ks/Documents/research/projD2/tmp_plots1')
parser.add_argument('--postfix_name', type=str, default='')
parser.add_argument('--disable_pad_token', action='store_true')
parser.add_argument('--root_data_dir', type=str, default='/Users/ks/Documents/research/projD2/function_vectors/dataset_files' if DEBUG_FLAG else 'function_vectors_v2/dataset_files')
# parser.add_argument('--dataset_name', type=str, default='antonym_len_8_1')
parser.add_argument('--dataset_name', type=str, default='prev_item')
parser.add_argument('--n_shots_icl', type=int, default=3)
parser.add_argument('--n_samples', type=int, default=5 if DEBUG_FLAG else 25)
parser.add_argument('--metric_to_use', type=str, default='f1_score')  # f1_score, exact_match_score, first_word_score
parser.add_argument('--prefixes', help='Prompt template prefixes to be used', type=json.loads, required=False, default={"input":"Q:", "output":"A:", "instructions":""})
parser.add_argument('--separators', help='Prompt template separators to be used', type=json.loads, required=False, default={"input":"\n", "output":"\n\n", "instructions":""})    
parser.add_argument('--device', type=str, default=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--check_if_result_already_exists', action='store_true')

_StoreTrueAction(option_strings=['--check_if_result_already_exists'], dest='check_if_result_already_exists', nargs=0, const=True, default=False, type=None, choices=None, required=False, help=None, metavar=None)

In [6]:
args, unknown = parser.parse_known_args()
print('unknown args:', unknown)
print('args:', args)

unknown args: ['--f=/Users/ks/Library/Jupyter/runtime/kernel-v2-58089lRDM6XES3E6R.json']
args: Namespace(model_name='gpt2', model_args='', out_folder='/Users/ks/Documents/research/projD2/tmp_plots1', postfix_name='', disable_pad_token=False, root_data_dir='/Users/ks/Documents/research/projD2/function_vectors/dataset_files', dataset_name='prev_item', n_shots_icl=3, n_samples=5, metric_to_use='f1_score', prefixes={'input': 'Q:', 'output': 'A:', 'instructions': ''}, separators={'input': '\n', 'output': '\n\n', 'instructions': ''}, device=device(type='cpu'), seed=42, check_if_result_already_exists=False)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(args.model_name)
if not args.disable_pad_token:
    print(f'adding pad token: {tokenizer.eos_token}')
    tokenizer.pad_token = tokenizer.eos_token
    
try:
    os.environ["TOKENIZERS_PARALLELISM"] = "true"  # not blocking, just to prevent warnings and faster tokenization
except:
    pass
padding_flag = tokenizer.pad_token_id is not None  # this is up to if the tokenizer was configured with padding or not

adding pad token: <|endoftext|>


In [8]:
device = torch.device(args.device)
print(f'Using device: {device} [cuda available? => {torch.cuda.is_available()}, cuda version: {torch.version.cuda}, args.device = "{args.device}"]')

Using device: cpu [cuda available? => False, cuda version: None, args.device = "cpu"]


In [9]:
model_extra_args = {}
for arg in args.model_args.split(','):
    if arg == '':
        continue
    k, v = arg.split('=')
    model_extra_args[k] = v
print(f'model_extra_args: {model_extra_args}')

model = AutoModelForCausalLM.from_pretrained(args.model_name, **model_extra_args).eval().requires_grad_(False).to(device)
model_aux = llm_utils.model_extra(model=model, device=device)
model_config = copy.deepcopy(model.config)
config = model_aux.config
# del model

n_embd = model_aux.n_embd
n_head = model_aux.n_head
head_size = model_aux.head_size
n_layer = model_aux.n_layer

pad_k = model_aux.pad_k
pad_v = model_aux.pad_v

# params_names_filter = opt_utils.get_filter_by_name(args.filter_layers)
params_names_filter = opt_utils.get_filter_by_name('attn_only')
# params_names_filter = lambda x: ('attn' in x or 'attention' in x) and 'weight' in x

# n_shots_icl = [int(x) for x in args.n_shots_icl.split(',')]
n_shots_icl = args.n_shots_icl
print(f'n_shots_icl: {n_shots_icl}')
metric_to_use = args.metric_to_use
n_samples = args.n_samples

model_extra_args: {}
Loading config from /Users/ks/Documents/research/projD2/opt07/bl2/clean_bu_code_v2/flow_graph_configs/config_gpt2.json
{'config_name': 'gpt2', 'layer_format': 'transformer.h.{}', 'layer_mlp_format': 'transformer.h.{}.mlp', 'layer_attn_format': 'transformer.h.{}.attn', 'ln1': 'transformer.h.{}.ln_1', 'attn_q': 'transformer.h.{}.attn.c_attn', 'attn_k': 'transformer.h.{}.attn.c_attn', 'attn_v': 'transformer.h.{}.attn.c_attn', 'attn_o': 'transformer.h.{}.attn.c_proj', 'ln2': 'transformer.h.{}.ln_2', 'mlp_ff1': 'transformer.h.{}.mlp.c_fc', 'mlp_ff2': 'transformer.h.{}.mlp.c_proj', 'mlp_act': 'transformer.h.{}.mlp.act', 'include_mlp_bias': True, 'include_attn_bias': True}
filter_name_or_code: attn_only --> filtering: going to change only attn layers
n_shots_icl: 3


In [10]:
print(model.config)

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.40.1",
  "use_cache": true,
  "vocab_size": 50257
}



In [11]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [12]:
# the prefix is a code for which version of the code we use in this notebook
# previously: f'av7.0_{args.model_name.replace("/", "-")}_{args.dataset_name}_[{args.n_shots_icl}]_{n_samples}_{metric_to_use}_{args.postfix_name}_'
output_prefix = f'cm8_{args.model_name.replace("/", "-")}_{args.dataset_name}_[{args.n_shots_icl}]_{n_samples}_{metric_to_use}_{args.postfix_name}_'

plot_wrapper = plot_aux_wrapper(output_folder=args.out_folder, 
                            output_prefix=output_prefix,
                            local_show=True)

show_every_n_layer = 1 if n_layer <= 12 else 2 if n_layer <= 24 else 4

In [13]:
is_llama = 'llama' in args.model_name or 'facebook/opt' in args.model_name
prepend_bos = not is_llama

prefixes=args.prefixes
separators=args.separators

compute_ppl=False
shuffle_labels=False
generate_str=False

In [14]:
dataset = exp_ra_utils.data_loading_wrapper(args.dataset_name, 
                                            root_data_dir=args.root_data_dir,
                                            seed=args.seed)

Load dataset: prev_item with 157 train, 21 valid and 47 test samples
Example: 1) {'input': 'OO', 'output': 'NN'}
Example: 2) {'input': 'oo', 'output': 'nn'}


In [15]:
sentence_example = exp_ra_utils.data_print_and_test_example(model, tokenizer=tokenizer,
                                                            dataset=dataset,
                                                            prepend_bos=prepend_bos,
                                                            prefixes=prefixes,
                                                            separators=separators,
                                                            shuffle_labels=shuffle_labels)

ICL prompt:
 '<|endoftext|>Q: i\nA: h\n\nQ: OO\nA: NN\n\nQ: oo\nA: nn\n\nQ: mm\nA: ll\n\nQ: r\nA: q\n\nQ: XVIII\nA:' 


Zero-Shot Prompt:
 '<|endoftext|>Q: XVIII\nA:'
Input Sentence: '<|endoftext|>Q: i\nA: h\n\nQ: OO\nA: NN\n\nQ: oo\nA: nn\n\nQ: mm\nA: ll\n\nQ: r\nA: q\n\nQ: XVIII\nA:' 

Input Query: 'XVIII', Target: 'XVII'

ICL Prompt Top K Vocab Probs:
 [(' o', 0.05167), (' O', 0.04458), (' n', 0.02428), (' Q', 0.02194), (' X', 0.02174)] 

Input Sentence: '<|endoftext|>Q: XVIII\nA:' 

Input Query: 'XVIII', Target: 'XVII'

Zero-Shot Prompt Top K Vocab Probs:
 [('\n', 0.06804), (' The', 0.06326), (' I', 0.05632), (' XV', 0.0369), (' What', 0.03228)] 



In [16]:
path_out_indirect_effect = f'{args.out_folder}/{output_prefix}_simple_casual_mediation_v2.pt'

meta_data = {
             'path_out_indirect_effect': path_out_indirect_effect,
             'example_sentence': sentence_example,  # to verify we ran what we wanted
             'ds_train_len': len(dataset['train']),
             'ds_valid_len': len(dataset['valid']),
             'ds_test_len': len(dataset['test']),
             'run_args': str(args.__dict__),
             'start_time': START_TIME,
             'debug_flag_on': DEBUG_FLAG}
meta_data_out = os.path.join(args.out_folder, f'{output_prefix}_meta_data_simple_indirect_effect_v2.json')
if args.check_if_result_already_exists and os.path.exists(meta_data_out):
    print(f'File already exists: {meta_data_out}. Terminating this run. If you wish to re-run, please remove the file or disable the flag --check_if_result_already_exists')
    print(f'Run args: {args}')
    print('Exit...')
    exit(0)

with open(meta_data_out, 'w') as f:
    json.dump(meta_data, f, indent=4)
    

In [17]:
def get_experiment_indirect_effect(example_index, split='train', annotate=False, dataset=dataset, curr_n_shots=n_shots_icl):
    res_effect = torch.zeros(n_layer, n_head).to(torch.float64)

    j = example_index
    if curr_n_shots == 0:
        word_pairs = {'input':[], 'output':[]}
    else:
        random_samples_without_current_j = np.random.choice(len(dataset[split]), curr_n_shots, replace=False)
        while j in random_samples_without_current_j:
            random_samples_without_current_j = np.random.choice(len(dataset[split]), curr_n_shots, replace=False)

        word_pairs = dataset[split][random_samples_without_current_j]
    word_pairs_test = dataset[split][j]
    if prefixes is not None and separators is not None:
        prompt_data = word_pairs_to_prompt_data(word_pairs, query_target_pair = word_pairs_test, prepend_bos_token=prepend_bos, 
                                                shuffle_labels=shuffle_labels, prefixes=prefixes, separators=separators)
    else:
        prompt_data = word_pairs_to_prompt_data(word_pairs, query_target_pair = word_pairs_test, prepend_bos_token=prepend_bos, shuffle_labels=shuffle_labels)
        
    # Get relevant parts of the Prompt
    query, target = prompt_data['query_target']['input'], prompt_data['query_target']['output']
    query = query[0] if isinstance(query, list) else query
    if generate_str:
        target = [target] if not isinstance(target, list) else target
    else:
        target = target[0] if isinstance(target, list) else target

    sentence = [create_prompt(prompt_data)]

    # Figure out tokens of interest
    target_token_id = get_answer_id(sentence[0], target, tokenizer)[0]

    device = model.device
    inputs = tokenizer(sentence, return_tensors='pt').to(device)
    original_pred_idx = len(inputs.input_ids.squeeze()) - 1

    # # if compute_nll:
    # target_completion = "".join(sentence + [target])
    # nll_inputs = tokenizer(target_completion, return_tensors='pt').to(device)
    # nll_targets = nll_inputs.input_ids.clone()
    # target_len = len(nll_targets.squeeze()) - len(inputs.input_ids.squeeze()) 
    # nll_targets[:,:-target_len] = -100  # This is the accepted value to skip indices when computing loss in nn.CrossEntropyLoss

    if annotate:
        tmp_print = sentence[0].replace("\n", "\\n")
        print(f'[{curr_n_shots}-shots] len={inputs.input_ids.shape[1]}, sentence_with_target={tmp_print}')
        print(f'target_token_id: {target_token_id}')

    clean_output = model(**inputs)
    clean_probs = torch.softmax(clean_output.logits[:,-1], dim=-1).squeeze(0).to(torch.float64)  # clean prob of the last position -> the answer we expect to check

    for layer_index in range(n_layer):
        Wo = llm_utils.rgetattr(model, config.attn_o.format(layer_index))
        original_Wo_weights = Wo.weight.clone()
        for head_index in range(n_head):
            tmp_Wo = original_Wo_weights.clone()
            if type(Wo) == Conv1D:
                tmp_Wo[head_size*head_index:head_size*(head_index+1)] = 0
            elif type(Wo) == nn.Linear:
                tmp_Wo[:, head_size*head_index:head_size*(head_index+1)] = 0
            else:
                raise ValueError(f'Unknown type of Wo: {type(tmp_Wo)} (currently only transformers.pytorch_utils.Conv1D and torch.nn.Linear are supported)')

            Wo.weight[:] = tmp_Wo

            indirect_output = model(**inputs)
            indirect_probs = torch.softmax(indirect_output.logits[:,-1], dim=-1).to(torch.float64)

            curr_indirect_effect = (indirect_probs - clean_probs)[0,target_token_id].squeeze().detach().cpu()
            if res_effect[layer_index, head_index] != 0:
                print(f'Warning: res_effect[{layer_index}, {head_index}] was not zero before the update. It was {res_effect[layer_index, head_index]}')
            res_effect[layer_index, head_index] = curr_indirect_effect.item()
        Wo.weight[:] = original_Wo_weights
    return res_effect


In [18]:
all_res = []

for j in range(n_samples):
    if j % 3 == 0:
        print(f'j={j}')
    curr_res = get_experiment_indirect_effect(example_index=j, annotate=j%3==0)
    all_res.append(curr_res)

j=0
[3-shots] len=36, sentence_with_target=<|endoftext|>Q: XIII\nA: XII\n\nQ: iii\nA: ii\n\nQ: tuesday\nA: monday\n\nQ: i\nA:
target_token_id: 289
j=3
[3-shots] len=34, sentence_with_target=<|endoftext|>Q: 25\nA: 24\n\nQ: CC\nA: BB\n\nQ: j\nA: i\n\nQ: mm\nA:
target_token_id: 32660


In [19]:
indirect_effect_res = torch.stack(all_res)
print(f'indirect_effect_res shape: {indirect_effect_res.shape} (n_examples, n_layer, n_head)')
print(f'Took mean result over {n_samples} samples')
mean_indirect_effect_res = indirect_effect_res.mean(dim=0)
print(f'mean_indirect_effect_res shape: {mean_indirect_effect_res.shape} (n_layer, n_head)')
torch.save(mean_indirect_effect_res, path_out_indirect_effect)

indirect_effect_res shape: torch.Size([5, 12, 12]) (n_examples, n_layer, n_head)
Took mean result over 5 samples
mean_indirect_effect_res shape: torch.Size([12, 12]) (n_layer, n_head)


In [20]:
print(f'DONE')

DONE
